# NLDAS example

The ETo/ETr calculation for recent NLDAS images have been updated to follow the equations in ASCE 2005.

Test that the precomputed ETr in the collection matches wih the ETr generated using the eerefet module. 

In [1]:
import ee
import openet.refetgee

ee.Initialize()

#### Load NLDAS for a single date

In [2]:
input_coll = ee.ImageCollection('NASA/NLDAS/FORA0125_H002')\
    .filterDate('2018-04-01', '2018-04-02')
    
# input_img = ee.Image(ee.ImageCollection(gridmet_coll.first()))
input_img = ee.Image('NASA/NLDAS/FORA0125_H002/A20180401_1800')

#### Ancillary data and test point

In [3]:
test_pnt = ee.Geometry.Point(-119.01, 39.01);

# Project latitude array to match source elevation grid exactly
elev_img = ee.Image("projects/eddi-noaa/nldas/elevation")
lat_img = ee.Image.pixelLonLat().select('latitude')\
    .reproject('EPSG:4326', elev_img.projection().getInfo()['transform'])
    
# Set the output crs and crsTransform to match the source images
# crs = input_img.projection().getInfo()['wkt']
crs = input_img.projection().getInfo()['crs']
geo = input_img.projection().getInfo()['transform']

#### Compute daily ETr from the aggregated hourly inputs

In [4]:
print(openet.refetgee.Daily.nldas(input_coll, elev=elev_img, lat=lat_img).etr\
      .reduceRegion(ee.Reducer.first(), test_pnt, crs=crs, crsTransform=geo)\
      .getInfo()['etr'])

7.774248853791551


#### Compute daily ETr as the sum of the hourly ETr

In [5]:
def hourly_etr(img):
    return ee.Image(openet.refetgee.Hourly.nldas(img, elev=elev_img, lat=lat_img).etr)\
        .copyProperties(img, ['system:time_start'])
etr = ee.Image(ee.ImageCollection(input_coll.map(hourly_etr)).sum())\
    .reduceRegion(ee.Reducer.first(), test_pnt, crs=crs, crsTransform=geo)
print(etr.getInfo()['etr'])

7.217448876991929
